In [1]:
print("ok")

ok


In [2]:
%pwd

'd:\\ilyass\\ptorjcts\\RAG\\Medical-chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\ilyass\\ptorjcts\\RAG\\Medical-chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [6]:
#Extract pdf

def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob = "*.pdf",
        loader_cls = PyPDFLoader
    )
    
    documents = loader.load()

    return documents

In [7]:
extracted_documents = load_pdf_file("data")

In [9]:
len(extracted_documents)

637

In [10]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs : List[Document]) -> List[Document]:
    
    minimal_docs : List[Document] = []

    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(Document(page_content=doc.page_content, metadata={"source": src}))

    return minimal_docs

In [11]:
minimal_docs = filter_to_minimal_docs(extracted_documents)

In [12]:
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [13]:
texts_chunk = text_split(minimal_docs)

In [14]:
print(len(texts_chunk))

5859


In [16]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings

embedding = download_embeddings()

C:\Users\HP 1030 G7\AppData\Local\Temp\ipykernel_2900\76838572.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
c:\Users\HP 1030 G7\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
len(vector)

384

In [35]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [ ]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

In [ ]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY

In [22]:
from pinecone import Pinecone

pinecone_api_key = PINECONE_API_KEY
pc = Pinecone(api_key=pinecone_api_key)


In [23]:
pc

In [24]:
from pinecone import ServerlessSpec

index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        index_name,
        dimension=384,
        metric="cosine",
        spec = ServerlessSpec(cloud = "aws",region="us-east-1")
    )

index = pc.Index(index_name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)


In [26]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding,
)

In [27]:
retriever = docsearch.as_retriever(search_type= "similarity", search_kwargs={"k":3})

In [28]:
retrieved_doc = retriever.invoke("What is Avne?")
retrieved_doc

[Document(id='aede69df-7dd4-4420-830b-e7eb7b694709', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='b45bc910-2927-44cb-90fa-074adceab887', metadata={'source': 'data\\Medical_book.pdf'}, page_content='OTHER\nSociety of USAF Flight Surgeons Online Catalog. <http://\nwww.sam.brooks.af.mil/ram/rammain.htm>.\nFederal Aviation Administration Office of Aviation Medicine.\n<http://www.faa.gov/avr/aamhome.htm>.\nNational Aeronautics and Space Administration Aerospace\nMedicine. <http://spacelink.msfc.nasa.gov>.\nMelissa Knopper\nAVM see Arteriovenous malformations\nAvoidant personality disorder see\nPersonality disorders\nAvulsions see Wounds\nAyur

In [38]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import os

chat = ChatOpenAI(
    model_name="qwen/qwen3-next-80b-a3b-instruct:free",
    openai_api_base="https://openrouter.ai/api/v1", 
)

msg = [HumanMessage(content="Say hi")]
resp = chat(msg)
print(resp.content)


AuthenticationError: Error code: 401 - {'error': {'message': 'No cookie auth credentials found', 'code': 401}}

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(chat, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly and gigantism are disorders caused by excess growth hormone from the pituitary gland. Gigantism occurs when this excess happens before bone growth stops in childhood, leading to unusual height, while acromegaly develops in adulthood, causing abnormal growth of bones and soft tissues. Both conditions are rare and often diagnosed late due to gradual symptom progression.


In [ ]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

Acne is a common skin disease characterized by pimples on the face, chest, and back, caused by clogged pores due to oil, dead skin cells, and bacteria. It is medically known as acne vulgaris and is the most prevalent skin condition in the United States, affecting nearly 17 million people.


In [ ]:
response = rag_chain.invoke({"input": "what is the Treatment of Acne?"})
print(response["answer"])

Treatment of acne depends on its severity: mild acne is treated with topical agents like tretinoin, benzoyl peroxide, or salicylic acid to reduce comedones; moderate to severe inflammatory acne may require topical or oral antibiotics, or isotretinoin for more persistent cases. Improvement typically takes weeks to months, and acne often recurs after stopping treatment.


In [ ]:
response = rag_chain.invoke({"input": "what dswithbappy?"})
print(response["answer"])

The term "dswithbappy" does not appear to be a recognized medical, scientific, or common term. It may be a typographical error or nonsense word. Please clarify or check the spelling for accurate assistance.
